In [1]:
from skopt import Optimizer
from skopt.learning import GaussianProcessRegressor
from skopt.learning.gaussian_process.kernels import RBF, ConstantKernel, Product
from tqdm import tqdm_notebook as tqdm
from skopt import gp_minimize
from time import sleep, time
import docker
import os
from pathlib import Path
import numpy as np

from get_borders import get_borders

/opt/conda/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


### Глобальные Параметры

In [2]:
# столько контейнеров вызываются для параллельной работы
batch_size = 10

# директория на сервере, хранит директории, которые будут монтироваться в контейнеры
folder_local = '/home/matyushinleonid/lhcb_ecal/summer/folder_local'

# директория для файлов input и output внутри контейнера
folder_container = '/home/nb_user/logs'

# python-клиент докера
client = docker.from_env()

# имя образа
container = "calorbuild"

# имена директорий, каждая соответствует своей копии образа
worker_names = ['worker_'+str(i) for i in range(1,batch_size+1)]
#worker_names = ['first_worker']

In [3]:
! rm -rf folder_local
! mkdir folder_local

for worker_name in worker_names:
    try:
        os.mkdir(Path(folder_local) / Path(worker_name))
    except Exception:
        pass

### Оптимизация (+ работа с контейнерами)

In [4]:
def write_input_file(worker_name, input_data):
    file_to_write = Path(folder_local) / Path(worker_name) / 'input.txt' #'{}/{}/input.txt'.format(folder_local, worker_name)
    
    y_inner,x_inner,a_inner = input_data
    result = get_borders(10,
                         16,
                         1,
                         y_inner,
                         x_inner,
                         a_inner)
    
    np.set_printoptions(threshold=np.inf, linewidth=np.inf)  # turn off summarization, line-wrapping
    with open(file_to_write, 'w') as f:
        np.savetxt(f, result, fmt="%i", delimiter=' ', newline='\n', header='', footer='', comments='# ')

def create_job(worker_name):
    folder_to_mount = '{}/{}'.format(folder_local, worker_name)
    client.containers.run(container,
                          privileged=True,
                          remove=True,
                          detach=True,
                          hostname='dev',
                          tty=True,
                          stdin_open=True,
                          volumes={folder_to_mount: {'bind': folder_container,
                                                     'mode': 'rw'}})

def read_output_file(worker_name):
    file_to_read = Path(folder_local) / Path(worker_name) / 'output.txt' #'{}/{}/output.txt'.format(folder_local, worker_name)
    with open(file_to_read, 'r') as myfile:
        data = myfile.read()
    return float(data)

def delete_output_file(worker_name):
    file_to_delete = Path(folder_local) / Path(worker_name) / 'output.txt'
    os.remove(file_to_delete)
    
def do_output_file_exists(worker_name):
    file_to_check = Path(folder_local) / Path(worker_name) / 'output.txt'
    if file_to_check.exists(): 
        return True
    else:
        return False

In [5]:
# start simulations
X = [[3, 16, 0.01],
     [3, 17, 0.563],
     [3, 18, 0.9475],
     [4, 12, 0.01],
     [4, 13, 0.5303],
     [4, 14, 0.9524],
     [5, 10, 0.3291],
     [5, 11, 0.7889],
     [6, 8, 0.01],
     [6, 9, 0.775]]

for i, worker_name in enumerate(worker_names):
    x = X[i]
    write_input_file(worker_name, x)
    create_job(worker_name)

In [6]:
# wait till simulation is done    
simulation_done_statuses = [False] * batch_size
time_before_sumulations = time()
while sum(simulation_done_statuses) < batch_size:
    for i, worker_name in enumerate(worker_names):
        if do_output_file_exists(worker_name):
            simulation_done_statuses[i] = True
    sleep(10)

    # sometimes docker doesn't produce output file. In this case we skip loop step
    if time() - time_before_sumulations > 20 * 60: 
        break

In [7]:
# perform an optimization step
final_answers = []
for i, worker_name in enumerate(worker_names):
    if simulation_done_statuses[i]:
        y = read_output_file(worker_name)
        delete_output_file(worker_name)
        final_answers.append({'params':X[i], 'metric':y})

In [8]:
final_answers

[{'metric': 20.569971, 'params': [3, 16, 0.01]},
 {'metric': 20.620939, 'params': [3, 17, 0.563]},
 {'metric': 20.6493, 'params': [3, 18, 0.9475]},
 {'metric': 19.813091, 'params': [4, 12, 0.01]},
 {'metric': 19.823282, 'params': [4, 13, 0.5303]},
 {'metric': 19.842154, 'params': [4, 14, 0.9524]},
 {'metric': 19.878986, 'params': [5, 10, 0.3291]},
 {'metric': 19.755478, 'params': [5, 11, 0.7889]},
 {'metric': 19.814251, 'params': [6, 8, 0.01]},
 {'metric': 19.950966, 'params': [6, 9, 0.775]}]